# B3 — Seleção de Fechamentos de Loop (Difícil)

Selecione um **conjunto independente de peso máximo** de candidatos a fechamento de loop (grafo de conflitos). Incentiva uso de QAOA/Ising; baseline guloso.

### Contexto
Em SLAM baseado em grafos, **candidatos a fechamento de loop** são correspondências entre o keyframe atual e poses passadas. Cada candidato adiciona uma restrição (aresta), porém hipóteses conflitantes (aliasing perceptual ou matches mutuamente inconsistentes) não podem ser aceitas juntas. Conflitos são modelados como arestas: nós = hipóteses de loop-closure, arestas = conflitos. Selecionar um subconjunto consistente vira um problema de **Maximum-Weight Independent Set (MWIS)** onde pesos codificam confiança geométrica ou ganho de informação. [`common.data_utils.mwis_loop_closure_instance`](common/data_utils.py) gera grafos de conflito e pesos de confiança.

### Por que quântico?
MWIS admite formulação Ising/QUBO com variáveis binárias $z_i\in\{0,1\}$ impondo penalidades de independência. Instâncias difíceis são combinatórias e altamente não convexas — terreno propício para QAOA ou heurísticas de annealing quântico. Amostragem quântica combinada a pós-processamento clássico pode diversificar subconjuntos de alto valor além do heurístico guloso.

## Especificação de Entrada e Saída
**Entrada:** `G (networkx.Graph)`, `w (np.ndarray)` pesos por nó  
**Saída:** `S (list[int])` índices do conjunto independente  
Orientações de recursos/runtime: qubits ≤ 12, passos do otimizador ≤ 250; manter tempo total ≤ ~10 s.

### Dicas de solução
- Monte o Hamiltoniano Ising de MWIS $H = -\sum_i w_i z_i + \lambda \sum_{(i,j)\in E} z_i z_j$ com penalidade $\lambda > \max_i w_i$.
- Use [`common.quantum_utils.default_device`](common/quantum_utils.py) com camadas customizadas ou QAOA para amostrar bitstrings; amostragem (shots) aproxima probabilidades.
- Repare bitstrings amostrados em conjuntos independentes viáveis (removendo conflitos pelos menores pesos marginais) e retenha o melhor valor.
- Truque híbrido: warm-start do QAOA com heurística gulosa ou refine a solução reparada com busca local clássica (trocas que melhoram peso).

## Setup

In [ ]:
import sys
import os
# Adiciona o diretório pai (qhacka-2025) ao caminho do Python
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))
import numpy as np
from common import data_utils as du
from common import baselines as bl
from common import quantum_utils as qu
from pennylane import numpy as pnp
import networkx as nx
np.random.seed(1337)

## Baseline (referência)

In [2]:
import networkx as nx
G, w = du.mwis_loop_closure_instance(n=40, p_edge=0.12, seed=0)
S = bl.baseline_mwis(G, w)
val = w[S].sum()
print("Valor MWIS baseline:", float(val))


Valor MWIS baseline: 16.057415452227122


## Sua tarefa: implementar `solve(...)`

In [ ]:
def solve(G, w):
    
    # --- Início da Solução QAOA Híbrida ---

    # 1. Importar bibliotecas
    import pennylane as qml
    from pennylane import numpy as pnp
    import networkx as nx
    from common import quantum_utils as qu
    from common import baselines as bl
    from pennylane.qaoa.layers import cost_layer, mixer_layer
    from pennylane.qaoa.mixers import x_mixer
    
    # 2. Função QAOA Otimizada
    def solve_component_qaoa_optimized(comp_G, comp_w_array, n_qubits):
        
        # --- 1. Construção do Hamiltoniano ---
        lambda_penalty = pnp.sum(comp_w_array) + 1.0
        cost_coeffs = []
        cost_ops = []
        z_coeffs = {i: 0.0 for i in range(n_qubits)}
        
        for i in range(n_qubits):
            z_coeffs[i] += comp_w_array[i] / 2.0
            
        for i, j in comp_G.edges():
            z_coeffs[i] -= lambda_penalty / 4.0
            z_coeffs[j] -= lambda_penalty / 4.0
            cost_coeffs.append(lambda_penalty / 4.0)
            cost_ops.append(qml.Z(i) @ qml.Z(j))

        for i, coeff in z_coeffs.items():
            if coeff != 0:
                cost_coeffs.append(coeff)
                cost_ops.append(qml.Z(i))
            
        cost_h = qml.Hamiltonian(cost_coeffs, cost_ops)
        mixer_h = x_mixer(wires=range(n_qubits))
        n_layers = 2
        
        # --- 2. Otimização dos Parâmetros ---
        dev_expval = qu.default_device(n_qubits=n_qubits)

        @qml.qnode(dev_expval)
        def circuit_expval(params):
            for i in range(n_qubits): qml.Hadamard(wires=i)
            for i in range(n_layers):
                cost_layer(params[i, 0], cost_h)
                mixer_layer(params[i, 1], mixer_h)
            return qml.expval(cost_h)

        lr = 0.01
        opt = qml.AdamOptimizer(stepsize=lr)
        params = pnp.array([[0.5, 0.8], [0.3, 0.7]], requires_grad=True)
        
        # Manter 10 passos
        for _ in range(10): 
            params = opt.step(circuit_expval, params)
        
        # --- 3. Amostragem com Parâmetros Otimizados ---
        shots = 600
        dev_sample = qu.default_device(n_qubits=n_qubits, shots=shots)
        
        @qml.qnode(dev_sample)
        def sample_circuit(params):
            for i in range(n_qubits): qml.Hadamard(wires=i)
            for i in range(n_layers):
                cost_layer(params[i, 0], cost_h)
                mixer_layer(params[i, 1], mixer_h)
            return qml.sample()

        samples = sample_circuit(params)
        
        if n_qubits == 1 and samples.ndim == 1:
            samples = pnp.reshape(samples, (len(samples), 1))

        # --- 5. Lógica de Reparo ---
        best_value = -1
        best_nodes_idx = []
        node_weights = {i: comp_w_array[i] for i in range(n_qubits)}

        for sample in samples:
            current_nodes = {i for i, bit in enumerate(sample) if bit == 1}
            
            conflicts_exist = True
            while conflicts_exist:
                conflicts_exist = False
                nodes_to_check = list(current_nodes)
                for u in nodes_to_check:
                    if u not in current_nodes: continue
                    for v in comp_G.neighbors(u):
                        if v in current_nodes: 
                            conflicts_exist = True
                            if node_weights[u] < node_weights[v]:
                                current_nodes.remove(u)
                                break
                            else:
                                current_nodes.remove(v)
                    if conflicts_exist:
                        break 
            
            current_value = pnp.sum(comp_w_array[list(current_nodes)])
            
            if current_value > best_value:
                best_value = current_value
                best_nodes_idx = list(current_nodes)
        
        return best_nodes_idx

    # --- Fim da Função QAOA ---

    # 3. Lógica Principal (Refinamento Quântico)
    S_current = set(bl.baseline_mwis(G, w))
    nodes_to_check = list(G.nodes())
    pnp.random.shuffle(nodes_to_check) 

    for u in nodes_to_check:
        if u in S_current:
            S_conflict = {v for v in G.neighbors(u) if v in S_current}
            sub_nodes = {u} | S_conflict
        else:
            S_conflict = {v for v in G.neighbors(u) if v in S_current}
            sub_nodes = {u} | S_conflict
        
        n_sub = len(sub_nodes)
        
        if 1 < n_sub <= 12: 
            
            sub_nodes_list = list(sub_nodes)
            mapping = {old: new for new, old in enumerate(sub_nodes_list)}
            sub_G = G.subgraph(sub_nodes)
            G_q = nx.relabel_nodes(sub_G, mapping)
            
            w_q = pnp.array([w[node] for node in sub_nodes_list])
            
            qaoa_nodes_remapped = solve_component_qaoa_optimized(G_q, w_q, n_sub)
            
            S_qaoa = {sub_nodes_list[i] for i in qaoa_nodes_remapped}
            
            S_current = (S_current - sub_nodes) | S_qaoa

    return sorted(list(S_current))

# --- Fim da Solução QAOA Híbrida ---

## Testes públicos (rápidos)

In [12]:
G, w = du.mwis_loop_closure_instance(n=50, p_edge=0.10, seed=3)
S = solve(G, w)
# Verificação de viabilidade
for u, v in G.edges():
    assert not (u in S and v in S), "Não é independente"
val = w[S].sum()
val_base = w[bl.baseline_mwis(G, w)].sum()
print("Seu valor:", float(val), "Baseline:", float(val_base))
assert val >= 0.92 * val_base - 1e-6, "Muito abaixo do baseline"
print("OK")


Seu valor: 20.74151297714591 Baseline: 20.44907078626749
OK


> Testes adicionais serão executados pelos organizadores com seeds/tamanhos diferentes.